# Module 4: Introduction to Semantic Kernel Processes

## Building an AI Travel Agent with Semantic Kernel Processes

### Prerequisites

This example demonstrates the practical application of Semantic Kernel Processes concepts. Think of processes as like building a big node-based graphed connected by business "processes"

We'll build a travel agent that showcases the key features of processes
while implementing real-world functionality.


## Key Concepts

1. **Process Steps**: The travel agent uses multiple steps (intro, user input, response)
2. **Event Handling**: Communication between steps via defined events
3. **State Management**: Maintaining conversation and travel preference state
4. **AI Integration**: Context-aware responses using OpenAI

In [16]:
import asyncio
from enum import Enum
from typing import ClassVar

from pydantic import Field

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.chat_completion_client_base import (
    ChatCompletionClientBase,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel_pydantic import KernelBaseModel
from semantic_kernel.processes.kernel_process.kernel_process_step import (
    KernelProcessStep,
)
from semantic_kernel.processes.kernel_process.kernel_process_step_context import (
    KernelProcessStepContext,
)
from semantic_kernel.processes.kernel_process.kernel_process_step_state import (
    KernelProcessStepState,
)
from semantic_kernel.processes.local_runtime.local_event import KernelProcessEvent
from semantic_kernel.processes.local_runtime.local_kernel_process import start
from semantic_kernel.processes.process_builder import ProcessBuilder

## 2. Define Events and State Models

Following best practices from Section 5 of the module:
- Use clear, descriptive event names in an Enum
- Keep state minimal and focused
- Use proper typing for state objects

In [17]:
class TravelAgentEvents(Enum):
    """
    Events that control the process flow.
    Following the best practice of grouping related events in Enums.
    """

    StartProcess = "startProcess"
    IntroComplete = "introComplete"
    UserInputReceived = "userInputReceived"
    AgentResponseGenerated = "agentResponseGenerated"
    Exit = "exit"


class TravelPreferencesState(KernelBaseModel):
    """
    State for managing user inputs.
    Demonstrates the 'State Management' concept from Section 2.3.
    """

    user_inputs: list[str] = []
    current_input_index: int = 0


class TravelAgentState(KernelBaseModel):
    """
    State for tracking conversation and travel details.
    Shows how to maintain complex state across executions.
    """

    chat_messages: list = []
    destination: str = ""
    budget: str = ""
    duration: str = ""

## 3. Create the Introduction Step

This implements a basic process step as described in Section 2.1.
Steps are the building blocks of processes, capable of:
- Handling events
- Executing functions
- Emitting events

In [18]:
class TravelAgentIntroStep(KernelProcessStep):
    @kernel_function
    async def print_intro_message(self):
        print("""
🌍 Welcome to your AI Travel Agent! 🧳
I'm here to help plan your perfect trip.
Let's discuss your travel preferences.
        """)


## 4. Implement User Input Step

This step demonstrates:
- Stateful components (Section 4.2)
- Event handling (Section 2.2)
- Error handling (Section 5.3)

In [19]:
class TravelUserInputStep(KernelProcessStep[TravelPreferencesState]):
    GET_USER_INPUT: ClassVar[str] = "get_user_input"

    def create_default_state(self) -> "TravelPreferencesState":
        return TravelPreferencesState()

    def populate_user_inputs(self):
        """
        Populate with example user inputs for demonstration.
        In a real application, these would come from user interaction.
        """
        if self.state is not None:
            self.state.user_inputs = [
                "I want to plan a trip to Japan",
                "My budget is $5000",
                "I want to stay for 10 days",
                "What activities do you recommend?",
                "exit",
            ]

    async def on_activate(self):
        self.populate_user_inputs()

    async def activate(self, state: KernelProcessStepState[TravelPreferencesState]):
        """
        Activation handling following best practices from Section 5.2
        - Initialize state properly
        - Use proper typing
        """
        state.state = state.state or self.create_default_state()
        self.state = state.state
        self.populate_user_inputs()

    @kernel_function(name=GET_USER_INPUT)
    async def get_user_input(self, context: KernelProcessStepContext):
        """
        Handles user input and emits appropriate events.
        Demonstrates event handling from Section 2.2.
        """
        if not self.state:
            raise ValueError("State has not been initialized")

        user_message = self.state.user_inputs[self.state.current_input_index]
        print(f"\nTRAVELER: {user_message}")

        if "exit" in user_message.lower():
            await context.emit_event(process_event=TravelAgentEvents.Exit, data=None)
            return

        self.state.current_input_index += 1
        await context.emit_event(
            process_event=TravelAgentEvents.UserInputReceived, data=user_message
        )

## 5. Implement Travel Agent Response Step

This step showcases:
- AI Integration (Section 7.2)
- Complex state management
- Context-aware responses

In [20]:
class TravelAgentResponseStep(KernelProcessStep[TravelAgentState]):
    GET_AGENT_RESPONSE: ClassVar[str] = "get_agent_response"
    state: TravelAgentState = Field(default_factory=TravelAgentState)

    async def activate(self, state: "KernelProcessStepState[TravelAgentState]"):
        """Initialize the agent's state following best practices."""
        self.state = state.state or TravelAgentState()
        self.state.chat_messages = self.state.chat_messages or []

    @kernel_function(name=GET_AGENT_RESPONSE)
    async def get_agent_response(
        self, context: "KernelProcessStepContext", user_message: str, kernel: "Kernel"
    ):
        """
        Generates context-aware responses using AI.
        Demonstrates AI Interactions use case from Section 7.2.
        """
        self.state.chat_messages.append({"role": "user", "message": user_message})

        # Update travel details based on user input
        message_lower = user_message.lower()
        if "plan a trip to" in message_lower:
            self.state.destination = message_lower.split("plan a trip to")[-1].strip()
        elif "budget" in message_lower:
            self.state.budget = message_lower.split("budget")[-1].strip()
        elif "stay for" in message_lower:
            self.state.duration = message_lower.split("stay for")[-1].strip()

        # Prepare context-aware prompt
        system_context = f"""You are an enthusiastic and knowledgeable AI Travel Agent.
Current trip details:
- Destination: {self.state.destination or 'Not specified'}
- Budget: {self.state.budget or 'Not specified'}
- Duration: {self.state.duration or 'Not specified'}

Provide helpful travel advice based on these details."""

        chat_service: ChatCompletionClientBase = kernel.get_service(
            service_id="default"
        )
        settings = chat_service.instantiate_prompt_execution_settings(
            service_id="default"
        )

        chat_history = ChatHistory()
        chat_history.add_system_message(system_context)
        chat_history.add_user_message(user_message)

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history, settings=settings
        )

        if response is None:
            raise ValueError(
                "Failed to get a response from the chat completion service"
            )

        answer = response[0].content
        print(f"\nTRAVEL AGENT: {answer}")

        self.state.chat_messages.append(answer)
        await context.emit_event(
            process_event=TravelAgentEvents.AgentResponseGenerated, data=answer
        )

## 6. Process Setup and Execution

This section demonstrates:
- Process building (Section 3)
- Event routing
- Process execution (Section 6)

In [21]:
async def run_travel_agent():
    """
    Sets up and runs the travel agent process.
    Shows how to:
    - Initialize the kernel
    - Configure process steps
    - Define event routing
    - Start the process
    """
    # Initialize the kernel with OpenAI service
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id="default"))

    # Build the travel agent process
    process = ProcessBuilder(name="TravelAgent")

    # Add process steps
    intro_step = process.add_step(TravelAgentIntroStep)
    user_input_step = process.add_step(TravelUserInputStep)
    agent_response_step = process.add_step(TravelAgentResponseStep)

    # Define the process flow using events
    process.on_input_event(event_id=TravelAgentEvents.StartProcess).send_event_to(
        target=intro_step
    )

    intro_step.on_function_result(
        function_name=TravelAgentIntroStep.print_intro_message.__name__
    ).send_event_to(target=user_input_step)

    user_input_step.on_event(event_id=TravelAgentEvents.Exit).stop_process()

    user_input_step.on_event(
        event_id=TravelAgentEvents.UserInputReceived
    ).send_event_to(target=agent_response_step, parameter_name="user_message")

    agent_response_step.on_event(
        event_id=TravelAgentEvents.AgentResponseGenerated
    ).send_event_to(target=user_input_step)

    # Build and start the process
    travel_agent_process = process.build()
    await start(
        process=travel_agent_process,
        kernel=kernel,
        initial_event=KernelProcessEvent(id=TravelAgentEvents.StartProcess, data=None),
    )

## 7. Run the Travel Agent

This example demonstrates a practical implementation of the concepts from Module 4,
showing how to build an AI-powered travel agent using Semantic Kernel Processes.

The implementation showcases:
- Process steps and event handling
- State management
- AI integration
- Best practices for process design

Execute this cell to start the travel agent process:

In [22]:
await run_travel_agent()


🌍 Welcome to your AI Travel Agent! 🧳
I'm here to help plan your perfect trip.
Let's discuss your travel preferences.
        

TRAVELER: I want to plan a trip to Japan
